## Processing of Yelp SQLite dataset

#### 1. Import required libraries:

In [1]:
# import framework to work with sqlite database in Python
import sqlite3
# pandas will be used for manipulating data sets retrieved from the database file
import pandas as pd
import unicodedata
# pickle will be used to serialise the prepared dataframe to be loaded later
import pickle

#### 2. Create connection to SQLite database file and specify what format to return objects in:

In [2]:
# set up connection to the database
connection = sqlite3.connect("yelpResData.db")
# create cursor object for interaction with the database
cur = connection.cursor()
# specify how to handle bytes in database
connection.text_factory = lambda x: str(x, 'utf-8', 'ignore')

# previously attempted text factories below
#connection.text_factory = bytes
#connection.text_factory = lambda x: str(x, 'iso-8859-1')

#### 3. Query the database to get all table names:

In [3]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('review',), ('restaurant',), ('reviewer',)]


#### 4. Query each table in the database and import the output into a pandas dataframe:

In [4]:
review_table = pd.read_sql_query('SELECT * FROM review', connection)
#restaurant_table = pd.read_sql_query('SELECT * FROM restaurant', connection)
reviewer_table = pd.read_sql_query('SELECT * FROM reviewer', connection)

#### 5. Normalise Unicode encoding in reviewContent column (e.g. "\xa0") in the review_table:

In [5]:
series = pd.Series(review_table['reviewContent'])
series = series.str.normalize("NFKD")
review_table['reviewContent'] = series.values

#### 6. Extract genuine ("N", i.e. not filtered) and filtered ("Y", i.e. filtered) reviews from dataset:

In [6]:
reviews = review_table[(review_table["flagged"] == 'Y') | (review_table["flagged"] == 'N')].reset_index(drop = True)

In [7]:
number_of_reviews = len(reviews)
number_of_genuine_reviews = (reviews["flagged"] == 'N').sum()   
number_of_filtered_reviews = (reviews["flagged"] == 'Y').sum()   

print("Number of reviews:", number_of_reviews)
print("Number of genuine reviews:", number_of_genuine_reviews)
print("Number of filtered reviews:", number_of_filtered_reviews)

Number of reviews: 67019
Number of genuine reviews: 58716
Number of filtered reviews: 8303


In [8]:
# check if there are any duplicate reviews
reviews["reviewID"].nunique()

# save file to csv for further inspection
#df.to_csv("dataframe.csv", encoding='utf-8', index=False)

67019

#### 7. Fix date column formatting and remove "Updated -" from date values

In [9]:
# identify indeces containing "Updated" to remove the strings
date_column = pd.Series(reviews["date"])
indeces = date_column.str.contains('updated', case=False, regex=True)
print(indeces.value_counts())

true_indeces = indeces[indeces == True].index

False    67016
True         3
Name: date, dtype: int64


In [10]:
# print values to fix
for index in range(0, len(true_indeces)):
    print(date_column.iloc[true_indeces[index]])

Updated - 2/20/2012
Updated - 4/13/2012
Updated - 1/8/2011


In [11]:
# "slice" the individual values to remove "Updated -"
for index in true_indeces:
    date_column.iloc[index] = date_column.iloc[index][10:]
    
# check that update is successful
indeces = date_column.str.contains('updated', case=False, regex=True)
print(indeces.value_counts())

False    67019
Name: date, dtype: int64


In [12]:
# make date formatting consistent in the date column of the dataframe
reviews["date"] = date_column
reviews["date"] = pd.to_datetime(reviews["date"])

In [13]:
# save as .csv file to check dates are correct in Excel
#reviews.to_csv("date_test.csv", encoding='utf-8', index=False)

In [14]:
# show count of reviews by year
years = reviews["date"].dt.year
print(years.value_counts())

2011    17011
2010    15492
2012    10904
2009    10778
2008     7172
2007     3852
2006     1441
2005      364
2004        5
Name: date, dtype: int64


#### Remove rows with empty reviews

In [15]:
# remove empty reviews
empty_review_index_list = [62005, 62792]
reviews = reviews.drop(reviews.index[empty_review_index_list])

reviews = reviews.sort_index()

#### Extract subset of columns for the reviews dataframe:

In [90]:
columns = [reviews["date"], reviews["reviewerID"], reviews["reviewContent"], reviews["flagged"]]
headers = ["date","reviewerID", "reviewContent", "flagged"]
reviews = pd.concat(columns, axis=1, keys=headers).reset_index(drop = True)

reviews

,date,reviewerID,reviewContent,flagged
0,2012-09-22,bNYesZ944s6IJVowOnB0iA,"Unlike Next, which we'd eaten at the previous ...",N
1,2012-09-22,TRKxLC3y-ZvP45e5iilMtw,Probably one of the best meals I've had ever. ...,N
2,2012-09-19,0EMm8umAqXZzyhxNpL4M9g,Service was impeccable. Experience and present...,N
3,2012-09-06,DlwexC7z88ymAzu45skODw,"The problem with places like this, given the e...",N
4,2012-09-09,kW2dk1CWihmh3g7k9N2G8A,I have no idea how to write my review - dining...,N
...,...,...,...,...
67012,2012-04-07,fUXNHG4x1AN9oPpBSpCtUw,I've passed this place a bunch of times on my ...,N
67013,2009-06-05,QQCm2oqSnnz24Z-yHeMtxQ,Rubino's is not really setup as a retail opera...,N
67014,2012-05-06,OXRKydSyzC35n2fNUnbopg,This place is amazing. The food is great and ...,N
67015,2011-01-08,NE8ULc362aFvnkTDgvZbcA,OK went back the next day as I was driving by ...,N


In [23]:
# temporary - get usefulCount, coolCount and funnyCount features from dataframe
columns = [reviews["usefulCount"], reviews["coolCount"], reviews["funnyCount"]]
headers = ["usefulCount","coolCount", "funnyCount"]
extra_features = pd.concat(columns, axis=1, keys=headers).reset_index(drop = True)

# pickle
extra_features.to_pickle("./extra_features.pkl")

#### Explore "anonymous" users 

In [22]:
# i.e., those not found in the reviewer table (from the exercise in the above cell)

# "anonymous" means ID in reviews table but NOT in reviewerID
anonymous_reviewers = []
has_profile = []

reviewers_in_table1 = pd.Series(list(reviews["reviewerID"].unique()))
reviewers_in_table2 = set(list(reviewer_table["reviewerID"]))

for reviewer in reviewers_in_table1:
    if(reviewer not in reviewers_in_table2):
        anonymous_reviewers.append(reviewer)
        has_profile.append(0)
    else:
        has_profile.append(1)

In [1]:
print("Number of unique reviewers in reviews table: ", len(reviews['reviewerID'].unique()))
print("Number of unique reviewers in reviewer table: ", 
      len(reviewer_table['reviewerID'].unique()))

print("Number of \"anonymous\" reviewers: ", len(anonymous_reviewers))
# e.g. reviewer ID "xMYPc5tzV2PSryKFK_y1PQ" is found in reviews but not reviewers table
print()
print("Examples of \"anonymous\" reviewers: ")
print("Number of reviews in reviews table with reviewer ID xMYPc5tzV2PSryKFK_y1PQ: ", len(reviews[reviews["reviewerID"] == "xMYPc5tzV2PSryKFK_y1PQ"]))
print("Number of reviews in reviewers table with reviewer ID xMYPc5tzV2PSryKFK_y1PQ: ", len(reviewer_table[reviewer_table["reviewerID"] == "xMYPc5tzV2PSryKFK_y1PQ"]))
print()
print("Number of reviews in reviews table with reviewer ID ciAaaK5kBPGM1y8CtkJtXQ: ", len(reviews[reviews["reviewerID"] == "ciAaaK5kBPGM1y8CtkJtXQ"]))
print("Number of reviews in reviewers table with reviewer ID ciAaaK5kBPGM1y8CtkJtXQ: ", len(reviewer_table[reviewer_table["reviewerID"] == "ciAaaK5kBPGM1y8CtkJtXQ"]))

NameError: name 'reviews' is not defined

In [106]:
reviews[reviews["reviewerID"] == "xMYPc5tzV2PSryKFK_y1PQ"]
# for this particular reviewer, we can see that they are not present in the reviewers' table, yet have posted 
# many times at different dates, from 2007-2009

,date,reviewerID,reviewContent,flagged,hasProfile
581,2008-05-16,xMYPc5tzV2PSryKFK_y1PQ,The meal was a 3.5 hour experience that had mo...,N,0
1422,2007-01-14,xMYPc5tzV2PSryKFK_y1PQ,I have been waiting forever to get encased mea...,N,0
3596,2009-03-22,xMYPc5tzV2PSryKFK_y1PQ,A definite guilty pleasure. Like someone wrote...,N,0
4399,2008-08-25,xMYPc5tzV2PSryKFK_y1PQ,My wife and I made it to Schwa last Friday eve...,N,0
8997,2007-01-05,xMYPc5tzV2PSryKFK_y1PQ,Go late or be prepared to wait. Seriously. We ...,N,0
11845,2007-09-04,xMYPc5tzV2PSryKFK_y1PQ,"Come for the pizza, stay for the beer. The piz...",N,0
15479,2009-01-27,xMYPc5tzV2PSryKFK_y1PQ,There isn't a lunch this side of my grandmothe...,N,0
21375,2008-03-28,xMYPc5tzV2PSryKFK_y1PQ,Trip to the Gage was great on each end and a l...,N,0
24895,2008-03-23,xMYPc5tzV2PSryKFK_y1PQ,My favorite Chinese in all of Chicago. For tho...,N,0
24932,2007-01-05,xMYPc5tzV2PSryKFK_y1PQ,Very solid Costa Rican food (who are we kiddin...,N,0


In [122]:
# check if any two groups of users do not have fake reviews
x1 = len(reviews[(reviews['hasProfile'] == 1) & (reviews['flagged'] == 'Y' )])
x2 = len(reviews[(reviews['hasProfile'] == 1) & (reviews['flagged'] == 'N' )])
print("Proportion of fake reviews by reviewers with a profile: ")
print(x1 / (x1 + x2))
print("Proportion of genuine reviews by reviewers with a profile: ")
print(x2 / (x1 + x2))
print("Total: ", x1 + x2)
print()
x3 = len(reviews[(reviews['hasProfile'] == 0) & (reviews['flagged'] == 'Y' )])
x4 = len(reviews[(reviews['hasProfile'] == 0) & (reviews['flagged'] == 'N' )])
print("Proportion of fake reviews by reviewers without a profile: ")
print(x3 / (x3 + x4))
print("Proportion of genuine reviews by reviewers without a profile: ")
print(x4 / (x3 + x4))
print("Total: ", x3 + x4)

Proportion of fake reviews by reviewers with a profile: 
0.23015284166790326
Proportion of genuine reviews by reviewers with a profile: 
0.7698471583320967
Total:  26956

Proportion of fake reviews by reviewers without a profile: 
0.05234517361024438
Proportion of genuine reviews by reviewers without a profile: 
0.9476548263897556
Total:  40061


In [94]:
# add column in reviews table to indicate whether a reviewer is "anonymous"
reviews["hasProfile"] = has_profile

#### Pickle (serialise) the prepared dataframe objects to be loaded later

In [100]:
reviews.to_pickle("./reviews.pkl")

#### Close cursor and connection to database after use:

In [46]:
cur.close()
connection.close()